In [304]:
import os
import scanpy as sc
import anndata as ad
import squidpy as sq
import matplotlib.pyplot as plt
from ipywidgets import interact
import seaborn as sns
import SpatialDE
import pandas as pd
import numpy as np
import scvi
import warnings
from matplotlib.colors import ListedColormap
from matplotlib import cm


from skimage.segmentation import find_boundaries

In [4]:
sc.settings.set_figure_params(dpi=240, facecolor='white', frameon=False)
sc.settings.njobs = 32
plt.rcParams['figure.dpi'] = 240
sns.set_style("ticks")

In [5]:
RESOURCES_FOLDERNAME = "/home/dchafamo/final/data/"
RESULTS_FOLDERNAME = "/home/dchafamo/final/results/"
AUXILLIARIES_FOLDERNAME = "/home/dchafamo/final/tmp/"
FIGURES_FOLDERNAME = "/home/dchafamo/final/results/figures/"

In [6]:
sc.settings.figdir = FIGURES_FOLDERNAME

In [7]:
adata = sc.read_h5ad(os.path.join('/home/dchafamo/cHL_spatial/data/', 'cosmx_cHL.h5ad'))

/home/dchafamo/anaconda3/envs/tensor/lib/python3.9/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(


In [8]:
adata.obs.cell_types_level_3.value_counts()

CD4_T_cells          62175
B_cells              47451
Fibroblasts          38642
Mono/macs            34008
NK_cells             27064
Plasma_cells         22900
CD8_T_cells          21212
BEC                  20111
mDC                  12345
Epithelium_KRT16+    12092
Tumor                10759
LEC                   8683
Name: cell_types_level_3, dtype: int64

***
***
    Vizualize raw morphology images
***
***

In [9]:
adata.obs['cell_id_full'] = adata.obs.index
adatas = {}
for i, slide in enumerate(['Slide1', 'Slide2', 'Slide3']):
    ad = sq.read.nanostring(
        path=f"data/{slide}", 
        counts_file=f"/home/dchafamo/cHL_spatial/data/{slide}/SMI-0026_{slide}_exprMat_file.csv",
        meta_file=f"/home/dchafamo/cHL_spatial/data/{slide}/SMI-0026_{slide}_metadata_file.csv", 
        fov_file=f"/home/dchafamo/cHL_spatial/data/{slide}/SMI-0026_{slide}_fov_positions_file.csv"
    )

    ## Nanostring images are flipped on y axis
    ad.obsm['spatial'][:,1] = max(ad.obsm['spatial'][:,1]) + 20 - ad.obsm['spatial'][:,1]
    
    for fov in ad.uns["spatial"].keys():
        ad.uns["spatial"][fov]['images']['hires'] = \
            ad.uns["spatial"][fov]['images']['hires'].reshape(
                list(ad.uns["spatial"][fov]['images']['hires'].shape) + [1]
            ) 
        ## add segmentation boundary
        b = find_boundaries(ad.uns['spatial'][fov]['images']['segmentation'])
        ad.uns['spatial'][fov]['images']['segmentation'][b] = -1
      
    ## Transfer metadata 
    ad.obs['cell_id_full'] = str(i+1) + '_' + ad.obs.index
    new_obs = ad.obs.merge(adata.obs, how='left', on='cell_id_full')
    new_obs.index = new_obs.cell_id_full
    ad.obs = new_obs
    ad.obs['fov'] = ad.obs.fov_x.astype(str)
    ad.obs['cell_ID'] = ad.obs.cell_ID_x
    ad.obs['cell_types_level_3'] = ad.obs['cell_types_level_3'].astype(str)
    ad.obs.loc[ad.obs['cell_types_level_3'] == 'nan', 'cell_types_level_3'] = 'Unknown'
    
    sc.pp.filter_cells(ad, min_genes=10)
    sc.pp.normalize_total(ad, target_sum=1e4)
    sc.pp.log1p(ad)
    
    adatas[slide] = ad

In [305]:
binary = cm.get_cmap('binary', 128)
newcolors = np.vstack((binary(np.linspace(0, 0.5, 20)),binary(np.linspace(0.5, 1, 20))))
newcmp = ListedColormap(newcolors, name='binmap')
colors = ['#D4782F00','#462965FF','#C19CC5FF']
sns.color_palette(colors)

[(0.8313725490196079, 0.47058823529411764, 0.1843137254901961),
 (0.27450980392156865, 0.1607843137254902, 0.396078431372549),
 (0.7568627450980392, 0.611764705882353, 0.7725490196078432)]

In [306]:
def proximity_spatial(adatas, slide, gene, cell_type, fovs, threshold=0):
    adatas[slide].obs[gene] = f'{gene}-'
    adatas[slide].obs.loc[np.array(adatas[slide][:,gene].X.todense()).squeeze() > threshold, gene] = f'{gene}+'
    ad = adatas[slide][(adatas[slide].obs.fov.isin(fovs)) & 
                       (adatas[slide].obs.cell_types_level_3.isin([cell_type, 'Tumor'])),:].copy()
    ad.obs[f'celltype_{gene}'] = ad.obs[gene].astype(str) + ' ' + ad.obs['cell_types_level_3'].astype(str)
    ad.obs.loc[ad.obs.cell_types_level_3 == 'Tumor', f'celltype_{gene}'] = 'HRS'
    
    with warnings.catch_warnings() and plt.rc_context({'figure.figsize': (14, 8)}):
        warnings.simplefilter("ignore")
        sq.pl.spatial_segment(
            ad,
            color=f"celltype_{gene}",
            library_key="fov",
            library_id=fovs,
            seg_cell_id="cell_ID",
            seg_outline=True,
            img_cmap=newcmp,
            title='',
            palette=ListedColormap(colors),
            save=f'./chl_proximity_plots/{gene}/{gene}_' + f'{cell_type.replace("/","")}_{slide}_fov{fovs[0]}'.lower()
        )

In [307]:
cell_type = 'Mono/macs'
genes = ['CXCL10', 'CCL3', 'OSM', 'MMP12', 'IL1B', 'CCL8', 'CCL4', 'ICAM1', 'CXCL9']

In [308]:
chl_slide_fovs = ('Slide'+adata.obs.slide.astype(int).astype(str) + '_' + adata.obs.fov.astype(str)).unique()

In [ ]:
%%capture
for gene in genes:
    for slide in ["Slide1", "Slide2", "Slide3"]:
        for fov in adatas[slide].obs.fov.unique():
            if (slide + '_' + str(fov)) in chl_slide_fovs:
                proximity_spatial(adatas, slide, gene, cell_type, [fov])